# llamaindex 评估


In [1]:
import chromadb
from llama_index.core import (
    VectorStoreIndex,
    get_response_synthesizer,
    Settings,
)
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

In [2]:
# 设置嵌入模型和语言模型
Settings.embed_model = OllamaEmbedding(model_name="yxl/m3e:latest")
Settings.llm = Ollama(model="qwen2.5:0.5b", request_timeout=360)
# llm = Ollama(model="qwen2.5:0.5b", request_timeout=120.0)

In [3]:
# 初始化 Chroma 客户端，指定数据存储路径为当前目录下的 chroma_db 文件夹
db = chromadb.PersistentClient(path="C:/Users/Admin/Desktop/Data/diabetes/chroma_db")

# 获取或创建名为 "quickstart" 的集合，如果该集合不存在，则创建它
chroma_collection = db.get_or_create_collection("quickstart")

# 使用上述集合创建一个 ChromaVectorStore 实例，以便 llama_index 可以与 Chroma 集合进行交互
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# 创建一个存储上下文，指定向量存储为刚刚创建的 ChromaVectorStore 实例
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [4]:
# LlamaParse 是异步优先的，在 Notebook 中运行同步代码需要使用 nest_asyncio
import nest_asyncio

nest_asyncio.apply()
import os

os.environ["LLAMA_CLOUD_API_KEY"] = (
    "llx-YffHorRyQHp8qT3NEeZC5tHgXC6FEunNgwqozcLQhaGTGNPa"
)

In [5]:
from llama_parse import LlamaParse

parser = LlamaParse(
    result_type="markdown",
    language="ch_sim",
    verbose=True,
    num_workers=1,
)

documents = parser.load_data(
    [
        "C:/Users/Admin/Desktop/Data/diabetes/国家基层糖尿病防治管理指南.pdf",
        "C:/Users/Admin/Desktop/Data/diabetes/中国糖尿病护理及教育指南.pdf",
        "C:/Users/Admin/Desktop/Data/diabetes/中国血糖监测临床应用指南（2021）.pdf",
    ]
)

Parsing files:   0%|          | 0/3 [00:00<?, ?it/s]

Parsing files: 100%|██████████| 3/3 [02:05<00:00, 41.82s/it]


In [6]:
print(documents[0].text[:1000])
print(documents[0].get_content()[1000:10000])

# 国家基层糖尿病防治管理指南（2022）

# 主编 贾伟平

# 中华医学会糖尿病学分会

# 国家基层糖尿病防治管理办公室

# 上海交通大学医学院附属第六人民医院

组织编写



In [ ]:
# 读取文档
# documents = SimpleDirectoryReader("C:/Users/Admin/Desktop/Data/diabetes/").load_data()

# 构建索引
try:
    index = VectorStoreIndex.from_documents(
        documents=documents,
        storage_context=storage_context,
        transformations=[SentenceSplitter(chunk_size=256)],
    )
    print("Index created successfully")
except Exception as e:
    print("Error during index creation:", e)

Index created successfully


In [8]:
# 配置检索器
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,  # 返回最相似的前 n 个文档片段
)

# 配置响应合成器
response_synthesizer = get_response_synthesizer()

# 组装查询引擎
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [10]:
# 执行查询
response = query_engine.query("糖尿病诊断")
print(response)  # 输出查询结果

糖尿病的诊断通常包括以下几个步骤：

1. **自我监测**：患者会记录血糖水平、糖化血红蛋白等指标，以便实时了解自己的血糖变化趋势。
2. **就医检查**：医生会根据患者的症状和体征进行初步评估，并可能要求做一系列检查，如空腹血糖测试、随机血糖测试、尿液检查等。

如果出现糖尿病的任何症状，比如频繁的饥饿感或疲劳、多饮或多尿、体重下降或其他不适表现，则应及时就医。医生可能会建议你做一些进一步的检查，例如：

- **口服葡萄糖耐量试验（OGTT）**：通过摄入含糖食物后测血糖水平。
- **随机血糖测试**：即在未进行任何其他活动的情况下立即检测血糖。

以上信息供您参考，并不是必须全部掌握的内容。


In [9]:
chat_engine = index.as_chat_engine()

'糖尿病的诊断通常需要以下几个步骤：\n\n1. **病史询问**：医生会首先询问你的症状，如频繁尿频、体重增加、眼睛干涩、疲劳等。医生还会询问你是否有家族史。\n\n2. **体格检查**：医生会对你的身体进行全面检查，包括测量血压、血糖水平等。\n\n3. **实验室检测**：实验室测试，如空腹血糖和糖化血红蛋白（HbA1c）检测，可以帮助诊断糖尿病。\n\n4. **其他辅助检查**：有时可能需要进行胃镜、心电图或其他特定的检查来排除其他可能导致类似症状的原因。\n\n如果你有具体的症状或担心自己是否患有糖尿病，请尽快咨询医生。'

In [ ]:
respon = chat_engine.chat("糖尿病如何诊断")
respon.response.strip()

In [ ]:
while True:
    question = input("请输入您的问题（输入 '/bye' 以结束）：")
    if question.lower() == "/bye":
        break
    response = chat_engine.chat(question)
    print(response.response.strip())

评估 RAG 应用需要用到几个评估实体，分别是：

- Question: 指用户输入的问题，RAG 应用通过问题检索到相关的文档上下文
- Context: 指检索到的文档上下文，RAG 应用检索到相关文档后会将这些上下
  文结合用户问题一起提交给 LLM，最后生成答案
- Answer: 指生成的答案，RAG 应用将问题和上下文提交给 LLM 后，LLM 会
  根据这些信息来生成答案
- Grouth Truth: 指人工标注的正确答案，利用这个实体可以对生成的答案进
  行分析，从而得到评估结果，在 LlamaIndex 中，这个实体叫做 Reference Answer

其中 Question 和 Ground Truth 通过用户提供，Context 通过检索得到，Answer
是由 LLM 生成，后面我们在讲解的时候会沿用这些实体名称。在 LlamaIndex 中提供
了生成测试数据集的功能，可以帮助我们快速生成测试数据集，无需人工干预。


# 评估问题

## 适合微调的问题：

这些问题的答案相对固定，基于已知的医学知识，适合通过微调模型来优化回答。

糖尿病的分期是什么？

这是一个定义性问题，答案基于标准医学分类。

糖尿病患者应该多久进行一次散瞳眼检？

这是基于医疗指南的建议，答案相对固定。

低血糖有哪些症状？

低血糖症状是已知的，适合通过微调来提供准确回答。

糖尿病可以被逆转吗？

这是一个普遍关注的问题，答案基于当前医学知识。

连续血糖监测（CGM）在糖尿病管理中的好处是什么？

CGM 的好处是已知的，适合提供标准化回答。

压力如何影响糖尿病患者的血糖水平？

一般机制已知，适合微调模型来解释。

胰岛素在 2 型糖尿病管理中的作用是什么？

胰岛素的作用是明确的，适合提供标准回答。

糖尿病友好的饮食有哪些关键组成部分？

饮食建议是标准化的，适合微调模型来提供。

糖尿病教育如何改善患者的结果？

教育的好处是已知的，适合提供固定答案。

1 型糖尿病和 2 型糖尿病有什么区别？

这是一个基本的区分问题，答案固定。

## 适合 RAG 的问题：

这些问题可能需要最新的研究进展或个性化建议，适合用 RAG 来补充外部信息。

妊娠期糖尿病控制不良有哪些风险？

可能涉及最新的研究或统计数据。

运动如何影响胰岛素敏感性？

可能需要最新的研究结果。

糖尿病的长期并发症有哪些？

可能涉及新兴的研究或更新的数据。

妊娠糖尿病是如何诊断的？

诊断标准可能随时间更新。

糖尿病药物的最新发展是什么？

需要最新的药物信息。

## 适合两者结合的问题：

这些问题既需要基础的医学知识，又可能需要最新的研究成果，适合结合微调和 RAG 来回答。

压力如何影响糖尿病患者的血糖水平？

结合已知机制和最新研究。

运动如何影响胰岛素敏感性？

结合标准知识和最新研究成果。

糖尿病的长期并发症有哪些？

结合已知并发症和新兴研究。

妊娠糖尿病是如何诊断的？

结合标准诊断流程和最新指南。


In [29]:
from llama_index.core.evaluation import BatchEvalRunner
from llama_index.core.evaluation import ContextRelevancyEvaluator
from llama_index.core.evaluation import AnswerRelevancyEvaluator
from llama_index.core.evaluation import FaithfulnessEvaluator
from llama_index.core.evaluation import CorrectnessEvaluator
from llama_index.core.evaluation import PairwiseComparisonEvaluator
from llama_index.core.evaluation import RelevancyEvaluator

answer_relevancy_evaluator = AnswerRelevancyEvaluator()
context_relevancy_evaluator = ContextRelevancyEvaluator()
relevant_evaluator = RelevancyEvaluator()
correctness_evaluator = CorrectnessEvaluator()
faithfulness_evaluator = FaithfulnessEvaluator()
pairwiseComparisonEvaluator = PairwiseComparisonEvaluator()

runner = BatchEvalRunner(
    evaluators={
        "answer_relevancy": answer_relevancy_evaluator,
        "context_relevancy": context_relevancy_evaluator,
        "relevancy": relevant_evaluator,
        "correctness": correctness_evaluator,
        "faithfulness": faithfulness_evaluator,
        "pairwiseComparisonEvaluator": pairwiseComparisonEvaluator,
    },
    workers=8,
)

In [ ]:
node_parser = SentenceSplitter()
nodes = node_parser.get_nodes_from_documents(documents)
vector_index = VectorStoreIndex(nodes)
engine = vector_index.as_query_engine()
response = engine.query(question)
answer = str(response)

In [ ]:
questions = [example.query for example in examples]
ground_truths = [example.reference_answer for example in examples]
metrics_results = runner.evaluate_queries(
    engine, queries=questions, reference=ground_truths
)

for metrics in metrics_results.keys():
    print(f"metrics: {metrics}")
    eval_results = metrics_results[metrics]
    for eval_result in eval_results:
        print(f"score: {eval_result.score}")
        print(f"feedback: {eval_result.feedback}")
        if eval_result.passing is not None:
            print(f"passing: {eval_result.passing}")